### Wet-Chicken problem BNN_LV with PYMC3
Using NUTS in pymc3 to sample from posterior of BNN_LV

Samples saved out: `saved_samples/chicken_samples_pymc3.npy`

In [1]:
from autograd import numpy as np
from autograd import grad
from autograd.misc.optimizers import adam, sgd
from autograd import scipy as sp
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import sys
import pymc3 as pm
import theano.tensor as tt
from IPython.core.debugger import set_trace
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')
from utils.data_gen import sample_gaussian_mixture, generate_regression_outputs
from utils.BNN_pymc3 import BNN_LV as BNN_LV_pymc3
from utils.custom_callbacks_pymc3 import plot95ci, wb_scatter, build_wb_callback

## Wet Chicken

In [3]:
from utils.games import WetChicken2D

### Set up simulation and extract data

In [4]:
env = WetChicken2D(L=5, W=3, max_steps=20, seed=207)
no_action_policy = lambda state: (0,0)  # For any state, play the "do nothing" action.
random_policy = None  # If the policy is None, the simulator chooses an action at random.

# choose to run on the no-action policy
env.run(episodes=100, progress=100, policy=no_action_policy)

Episode 100/100 took 4 steps.


In [5]:
transitions = env.extract_transition_dataset()
transitions = transitions.sample(frac=1, replace=False, random_state=123)
transitions

,start_x,start_y,action_x,action_y,result_x,result_y
344,1,5,0,0,1,5
482,3,4,0,0,3,0
78,3,4,0,0,3,1
185,1,5,0,0,1,5
261,1,5,0,0,1,5
...,...,...,...,...,...,...
98,1,5,0,0,1,5
322,1,5,0,0,1,5
382,1,5,0,0,1,5
365,2,2,0,0,2,0


In [6]:
# Build training data (ignore X dimension and try to use starting Y position and Y action to predict landing Y position):
X_train = transitions[['start_x','start_y','action_x','action_y']].to_numpy()
Y_train = transitions[['result_x','result_y']].to_numpy()

print('X :',X_train.shape)
print('Y :',Y_train.shape)


X : (845, 4)
Y : (845, 2)


### Setup BNN_LV architecture and perform MLE fit

In [7]:
# Parameters
gamma_chicken = 1
sigma_chicken = 1

architecture_chicken = {'input_n':4, 
             'output_n':2, 
             'hidden_layers':[20,20],
             'biases' : [1,1,1],
             'activations' : ['relu', 'relu', 'linear'],
             'gamma':[gamma_chicken],
             'sigma':[sigma_chicken,sigma_chicken]}

bnn_lv_chicken = BNN_LV_pymc3(architecture=architecture_chicken)

bnn_lv_chicken.fit(X_train, Y_train, step_size=0.01, max_iteration=5000, check_point=500, regularization_coef=None)

Iteration 0 lower bound 10261.442453691097; gradient mag: 21343.807468401326
Iteration 500 lower bound 1.2855019894939093; gradient mag: 4.454922543872159
Iteration 1000 lower bound 0.6725601176985965; gradient mag: 1.4737438678030916
Iteration 1500 lower bound 0.4625975141858506; gradient mag: 0.8132697922240075
Iteration 2000 lower bound 0.3830472018307709; gradient mag: 0.482031395087857
Iteration 2500 lower bound 0.34539502088699414; gradient mag: 0.2856555371384487
Iteration 3000 lower bound 0.3267464129270182; gradient mag: 0.17875239396716047
Iteration 3500 lower bound 0.3177072434747046; gradient mag: 0.10119351185466771
Iteration 4000 lower bound 0.3138387209103612; gradient mag: 0.05704341375316368
Iteration 4500 lower bound 0.3121992429559065; gradient mag: 0.05813184279935818


### Prepare parameters for pymc3 sampling

In [ ]:
p_mu = 0
p_sigma = 5
l_sigma = 0.25
lv_gamma = 2 # usual, multiplied by 2

my_tune = 500
my_draws = 400
my_target_accept = 0.9
my_max_treedepth = 15

### Run pymc3

In [9]:
with pm.Model() as pm_model:
    
    # Prior on w (same shape as MLE)
    w_prior = pm.Normal('w', mu=p_mu, sigma=p_sigma, shape=bnn_lv_chicken.weights.shape) 
    
    # Latent variable prior (same shape as number of datapoints)
    lv_prior = pm.Normal('z', mu=0, sigma=lv_gamma, shape=(X_train.shape[0],1))
        
    y = pm.Normal('y', mu = bnn_lv_chicken.forward(X = X_train, input_noise = lv_prior, weights=w_prior),
                       sigma = l_sigma,
                       observed = Y_train)
    
    trace = pm.sample(tune=my_tune, draws=my_draws, start = {'w': bnn_lv_chicken.weights},
                      target_accept = my_target_accept, max_treedepth = my_max_treedepth)

### Complete sampling and save/load samples obtained

In [4]:
# np.save('saved_samples/chicken_samples_pymc3.npy',trace['w'])
posterior_samples_pymc3_chicken = np.load('saved_samples/chicken_samples_pymc3.npy')

In [5]:
posterior_samples_pymc3_chicken.shape

(1600, 1, 582)